https://developers.google.com/gmail/api/quickstart/python

In [1]:
!hostname

9e9eb3710c81


# install modules

In [2]:
!pip install --upgrade google-api-python-client

  Using cached https://files.pythonhosted.org/packages/9a/b4/a955f393b838bc47cbb6ae4643b9d0f90333d3b4db4dc1e819f36aad18cc/google_api_python_client-1.8.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/63/7e/a523169b0cc9ce62d56e07571db927286a94b1a5f51ac220bd97db825c77/google_api_core-1.16.0-py2.py3-none-any.whl
  Found existing installation: google-api-python-client 1.7.6
    Uninstalling google-api-python-client-1.7.6:
      Successfully uninstalled google-api-python-client-1.7.6


In [3]:
!pip install --upgrade oauth2client

  Using cached https://files.pythonhosted.org/packages/95/a9/4f25a14d23f0786b64875b91784607c2277eff25d48f915e39ff0cff505a/oauth2client-4.1.3-py2.py3-none-any.whl
  Found existing installation: oauth2client 4.1.2
    Uninstalling oauth2client-4.1.2:
      Successfully uninstalled oauth2client-4.1.2


In [4]:
from __future__ import print_function
from googleapiclient.discovery import build
from httplib2 import Http
from oauth2client import file, client, tools

# authenticate our token

The following code cell intentionally prevents the rest of the notebook from automatically executing

In [ ]:
stop -- action needed

the script below must be run as a .py file in order to launch your browser and authenticate

The .py file contains

If the error message is

`raise HttpAccessTokenRefreshError(error_msg, status=resp.status)
oauth2client.client.HttpAccessTokenRefreshError: invalid_grant: Bad Request`
        
then delete "token.json"

# now that we've authenticated with our token, we can send email
https://developers.google.com/gmail/api/guides/sending

## message text needs to be base64 encoded

https://en.wikipedia.org/wiki/Base64

"Base64 is a group of binary-to-text encoding schemes that represent binary data in an ASCII string format by translating it into a radix-64 representation."

Why do we use base64 for SMTP? Answer: SMTP is so old that the number of characters is stricted. To expand the character set, base64 encode. https://www.quora.com/Why-are-MIME-attachments-encoded-in-base64 and https://superuser.com/questions/402193/why-is-base64-needed-aka-why-cant-i-just-email-a-binary-file

In [5]:
import base64
base64.b64encode(bytes('your string', 'utf-8'))

b'eW91ciBzdHJpbmc='

a function that creates a base64 encoded email

In [6]:
#https://docs.python.org/3/library/email.mime.html
from email.mime.text import MIMEText
from email import encoders
    
def create_message(sender, to, subject, message_text):
    """Create a message for an email.

    Args:
    sender: Email address of the sender.
    to: Email address of the receiver.
    subject: The subject of the email message.
    message_text: The text of the email message.

    Returns:
    An object containing a base64url encoded email object.
    """
    message = MIMEText(message_text)
    message['to'] = to
    message['from'] = sender
    message['subject'] = subject
    
    # https://docs.python.org/3/library/base64.html
    return {'raw': base64.urlsafe_b64encode(message.as_string().encode('utf-8')).decode("utf-8")}
    # https://stackoverflow.com/questions/606191/convert-bytes-to-a-string
    
    #return {'raw': base64.urlsafe_b64encode(bytes("None",'utf-8'))}
    
    # https://docs.python.org/3.4/library/email-examples.html
    #return {'raw': encoders.encode_base64(message.as_string())} 

In [7]:
msg=create_message(sender="benpayne@umbc.edu", 
                   to="ben.is.located@gmail.com", 
                   subject="a subjct", 
                   message_text="msg text from Python")
msg

{'raw': 'Q29udGVudC1UeXBlOiB0ZXh0L3BsYWluOyBjaGFyc2V0PSJ1cy1hc2NpaSIKTUlNRS1WZXJzaW9uOiAxLjAKQ29udGVudC1UcmFuc2Zlci1FbmNvZGluZzogN2JpdAp0bzogYmVuLmlzLmxvY2F0ZWRAZ21haWwuY29tCmZyb206IGJlbnBheW5lQHVtYmMuZWR1CnN1YmplY3Q6IGEgc3ViamN0Cgptc2cgdGV4dCBmcm9tIFB5dGhvbg=='}

In [8]:
from googleapiclient import errors

def send_message(service, user_id, message):
    """Send an email message.

  Args:
    service: Authorized Gmail API service instance.
    user_id: User's email address. The special value "me"
    can be used to indicate the authenticated user.
    message: Message to be sent.

  Returns:
    Sent Message.
    """
    try:
        message = (service.users().messages().send(userId=user_id, body=message)
                   .execute())
        print ('Message Id: %s' % message['id'])
        return message
    # https://stackoverflow.com/questions/42541857/error-using-gmail-api-tuto-using-python-3-except-errors-httperror-error
    except errors.HttpError as error:
        print ('An error occurred: %s' % error)

In [9]:
store = file.Storage('token.json')
creds = store.get()
service = build('gmail', 'v1', http=creds.authorize(Http()))

The following also works

# with attachment

In [10]:
from email.mime.audio import MIMEAudio
from email.mime.base import MIMEBase
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart
import mimetypes
import os

In [11]:
# https://developers.google.com/gmail/api/guides/sending#creating_messages_with_attachments
def create_message_with_attachment(
    sender, to, subject, message_text, file):
    """Create a message for an email.

  Args:
    sender: Email address of the sender.
    to: Email address of the receiver.
    subject: The subject of the email message.
    message_text: The text of the email message.
    file: The path to the file to be attached.

  Returns:
    An object containing a base64url encoded email object.
    """
    message = MIMEMultipart()
    message['to'] = to
    message['from'] = sender
    message['subject'] = subject

    msg = MIMEText(message_text)
    message.attach(msg)

    content_type, encoding = mimetypes.guess_type(file)

    if content_type is None or encoding is not None:
        content_type = 'application/octet-stream'
    main_type, sub_type = content_type.split('/', 1)
    if main_type == 'text':
        fp = open(file, 'rb')
        msg = MIMEText(fp.read(), _subtype=sub_type)
        fp.close()
    elif main_type == 'image':
        fp = open(file, 'rb')
        msg = MIMEImage(fp.read(), _subtype=sub_type)
        fp.close()
    elif main_type == 'audio':
        fp = open(file, 'rb')
        msg = MIMEAudio(fp.read(), _subtype=sub_type)
        fp.close()
    else:
        fp = open(file, 'rb')
        msg = MIMEBase(main_type, sub_type)
        msg.set_payload(fp.read())
        fp.close()
    filename = os.path.basename(file)
    msg.add_header('Content-Disposition', 'attachment', filename=filename)
    message.attach(msg)

#    return {'raw': base64.urlsafe_b64encode(message.as_string())}
    return {'raw': base64.urlsafe_b64encode(message.as_string().encode('utf-8')).decode("utf-8")}

# rename files

In [12]:
import glob
import hashlib
import time
import shutil
import random

In [13]:
list_of_grp1_files = glob.glob('grp1/*.ipynb')

In [14]:
len(list_of_grp1_files)

25

In [15]:
list_of_grp2_files = list_of_grp1_files#glob.glob('grp/*.ipynb')

In [16]:
list_of_grp2_files

['grp1/pb70051.ipynb',
 'grp1/jconroy1.ipynb',
 'grp1/cj86416.ipynb',
 'grp1/hlaper1.ipynb',
 'grp1/jliang5.ipynb',
 'grp1/ti83170.ipynb',
 'grp1/tev1.ipynb',
 'grp1/vjyesta1.ipynb',
 'grp1/am27.ipynb',
 'grp1/gmoummi1.ipynb',
 'grp1/ro30897.ipynb',
 'grp1/dbrown8.ipynb',
 'grp1/parthms1.ipynb',
 'grp1/pmourya1.ipynb',
 'grp1/hq16977.ipynb',
 'grp1/anudeek1.ipynb',
 'grp1/tris1.ipynb',
 'grp1/wfrenc1.ipynb',
 'grp1/kw35573.ipynb',
 'grp1/n98.ipynb',
 'grp1/tugcee1.ipynb',
 'grp1/zt71691.ipynb',
 'grp1/chandus1.ipynb',
 'grp1/bsama1.ipynb',
 'grp1/vl29331.ipynb']

In [17]:
#len(list_of_dice_files)

### demo of hash from string

In [18]:
hashlib.md5(b"Nobody inspects the spammish repetition").hexdigest()

'bb649c83dd1ea5c9d9dec9a18df0ffe9'

### we'll need a list of the student IDs later

In [19]:
list_of_student_ids = []
for file_name in list_of_grp1_files:
    user_name = file_name.split('/')[-1].split('.')[0]
    list_of_student_ids.append(user_name)

In [20]:
list_of_student_ids

['pb70051',
 'jconroy1',
 'cj86416',
 'hlaper1',
 'jliang5',
 'ti83170',
 'tev1',
 'vjyesta1',
 'am27',
 'gmoummi1',
 'ro30897',
 'dbrown8',
 'parthms1',
 'pmourya1',
 'hq16977',
 'anudeek1',
 'tris1',
 'wfrenc1',
 'kw35573',
 'n98',
 'tugcee1',
 'zt71691',
 'chandus1',
 'bsama1',
 'vl29331']

In [21]:
start_time = time.time()
collision = True
while collision:
    this_attempt = {}
    list_of_students1 = list_of_student_ids.copy()
    list_of_students2 = list_of_student_ids.copy()
    for stu_id in list_of_student_ids:
        #print('stu_id=',stu_id)
        #print(time.time()-start_time)
        author1 = stu_id
        while author1 == stu_id:
            author1 = random.choice(list_of_students1)
        list_of_students1.remove(author1)
        author2 = stu_id
        while author2 == stu_id:
            author2 = random.choice(list_of_students2)
        list_of_students2.remove(author2)
        if author1 != author2:
            collision = False
            this_attempt[stu_id] = (author1, author2)
        else:
            print('collision found',author1, author2)
            collision = True
            break
            

In [22]:
this_attempt

{'pb70051': ('vl29331', 'tris1'),
 'jconroy1': ('bsama1', 'gmoummi1'),
 'cj86416': ('hq16977', 'jliang5'),
 'hlaper1': ('vjyesta1', 'pb70051'),
 'jliang5': ('ti83170', 'zt71691'),
 'ti83170': ('tev1', 'dbrown8'),
 'tev1': ('zt71691', 'cj86416'),
 'vjyesta1': ('wfrenc1', 'hq16977'),
 'am27': ('jliang5', 'hlaper1'),
 'gmoummi1': ('anudeek1', 'chandus1'),
 'ro30897': ('kw35573', 'vl29331'),
 'dbrown8': ('ro30897', 'jconroy1'),
 'parthms1': ('n98', 'tev1'),
 'pmourya1': ('jconroy1', 'kw35573'),
 'hq16977': ('pmourya1', 'vjyesta1'),
 'anudeek1': ('tris1', 'ti83170'),
 'tris1': ('gmoummi1', 'anudeek1'),
 'wfrenc1': ('hlaper1', 'tugcee1'),
 'kw35573': ('chandus1', 'pmourya1'),
 'n98': ('dbrown8', 'am27'),
 'tugcee1': ('am27', 'wfrenc1'),
 'zt71691': ('tugcee1', 'ro30897'),
 'chandus1': ('parthms1', 'bsama1'),
 'bsama1': ('pb70051', 'n98'),
 'vl29331': ('cj86416', 'parthms1')}

In [23]:
store = file.Storage('token.json')
creds = store.get()
service = build('gmail', 'v1', http=creds.authorize(Http()))

In [26]:
folder_name = 'grp1'
for reviewer, author_tuple in this_attempt.items():
    for author in author_tuple:
        author_reviewer_hash = hashlib.md5(str.encode(author+reviewer)).hexdigest() # hash of student name
        print('reviewer =', reviewer,'; author = ', author,';',author_reviewer_hash)
        file_name = folder_name + '/' + author + '.ipynb'
        file_hash_name = folder_name + '/' + author_reviewer_hash + '.ipynb'
        #print('file_name=',file_name,'; file_hash_name=',file_hash_name)
        shutil.copy(file_name, file_hash_name)
        msg = create_message_with_attachment(
            'ben@com', # sender; doesn't seem to show up in actual email
#            'ben.is.located@gmail.com', # to - for testing 
            reviewer+'@umbc.edu',  # MASS MAIL - destination student
            '601 attempt to run this notebook: '+author_reviewer_hash, # subject
#            'Task: Review this notebook.\n'+
            'Task: attempt to run this notebook.\n'+
            'Reply to this email by 9:30pm on April 8 if you have feedback for the notebook author.\n\n'+
            'What changes (if any) to the code did you need to make?\n'+
#            'For example, is 
            'Was the documentation sufficient?\n'+
            'Were you able to get the code to run?\n'+
            'If no, where did you get stuck?\n'+
            'Did you encounter any errors?\n'+
            'Was the output correct?\n'+
#            'Are you able to understand the author\'s intent?\n'+
            'What suggestions do you have for the author?\n\n'+
            'Do not sign the reply email with your name.\n'+
            'Your comments should be in the email body. \n'+
            'Do not include any attachments in the reply email.', # body
            file_hash_name)

        send_message(service, "me", msg)  # send all the mails
        print('removing',file_hash_name)
        os.remove(file_hash_name)

reviewer = pb70051 ; author =  vl29331 ; 31e0a8adba026fc37cba64c1c957a742
Message Id: 1715bf9e63350dc9
removing grp1/31e0a8adba026fc37cba64c1c957a742.ipynb
reviewer = pb70051 ; author =  tris1 ; 18a838e678d31af3c72fb99e35bbf77a
Message Id: 1715bf9e83c38a37
removing grp1/18a838e678d31af3c72fb99e35bbf77a.ipynb
reviewer = jconroy1 ; author =  bsama1 ; 61d9b443c4f1e92ab5c8bb15d5bd654d
Message Id: 1715bf9ec110a65a
removing grp1/61d9b443c4f1e92ab5c8bb15d5bd654d.ipynb
reviewer = jconroy1 ; author =  gmoummi1 ; 43a01e45c1be45b3754953115662f9b7
Message Id: 1715bf9ec54d26ec
removing grp1/43a01e45c1be45b3754953115662f9b7.ipynb
reviewer = cj86416 ; author =  hq16977 ; 73bf30f1531d9f530481ba8be96e12b2
Message Id: 1715bf9f028a4c88
removing grp1/73bf30f1531d9f530481ba8be96e12b2.ipynb
reviewer = cj86416 ; author =  jliang5 ; 508298ab805cc81d0ce915cf25d44573
Message Id: 1715bf9f20861fbd
removing grp1/508298ab805cc81d0ce915cf25d44573.ipynb
reviewer = hlaper1 ; author =  vjyesta1 ; 99a8558b5901d82f6ce48d